In [1]:
from Microdados_CAGED import *
import pandas as pd

In [2]:
# Aqui cria-se dois data.frames: os dados brutos e os sumarizados pelo pandas
caged_microdados_full, caged_agrupamentos = importar_caged_mes_ano(ano_base=2023, mes_base=10)
caged_microdados = caged_microdados_full.query(f"uf == {piaui}")

KeyboardInterrupt: 

In [3]:
# Verificação se dos itens criados, temos de fato um data.frame
if isinstance(caged_agrupamentos["município"], pd.DataFrame):
    print("É um DataFrame")
else:
    print("Não é um DataFrame")

É um DataFrame


In [4]:
caged_agrupamentos["município"] # Verificando um dos itens do dicionário

Salario_admissão Salario_desligamento  Admissões  \
Período    município                                                    
2023-10-01 220005             1693.62              1848.30          5   
           220010             1321.38              1320.00          5   
           220020             1401.93              1420.54         29   
           220025             1328.33              1331.11          1   
           220027             1344.00                  NaN          1   
...                               ...                  ...        ...   
           221110             1123.01              1112.59         64   
           221120             1634.21              1752.78        311   
           221130             1548.51              1247.28         55   
           221140             1980.00              1980.00          0   
           221170             1694.81              1694.81          0   

                      Desligamentos  Saldo  
Período    município                        
2023-10-01 220005                 1      4  
           220010                 1      4  
           220020                40    -11  
           220025                 3     -2  
           220027                 0      1  
...                             ...    ...  
           221110               112    -48  
           221120               198    113  
           221130                30     25  
           221140                 1     -1  
           221170                10    -10  

[190 rows x 5 columns]

In [2]:
caged_dimensoes = importar_dimensoes() # o dicionário de tabelas-dimensões

NameError: name 'importar_dimensoes' is not defined

In [1]:
caged_dimensoes['uf']

NameError: name 'caged_dimensoes' is not defined

In [7]:
# O dicionário de dos dados agrupados, separados pelo respectivo grupo
# Esta etapa é apenas para separar os objetos, deixando salvos os objtos acima
 
dataframes_dict = {}
for key in pages[:-1]:
    dataframes_dict[key] = pd.DataFrame(caged_agrupamentos[key])

# temos um dicionário de DataFrames separados
dataframes_dict

{'município':                      Salario_admissão Salario_desligamento  Admissões  \
 Período    município                                                    
 2023-10-01 220005             1693.62              1848.30          5   
            220010             1321.38              1320.00          5   
            220020             1401.93              1420.54         29   
            220025             1328.33              1331.11          1   
            220027             1344.00                  NaN          1   
 ...                               ...                  ...        ...   
            221110             1123.01              1112.59         64   
            221120             1634.21              1752.78        311   
            221130             1548.51              1247.28         55   
            221140             1980.00              1980.00          0   
            221170             1694.81              1694.81          0   
 
                       

In [8]:
# Consolidação dos resultados, conforme os parâmetros da função que criamos
caged_formatado_completo = consolidar_caged(caged_agrupamentos, caged_dimensoes)

In [9]:
caged_formatado_completo

{'município':         Período           município  Admissões  Desligamentos  Saldo  \
 0    2023-10-01               Acauã          5              1      4   
 1    2023-10-01        Agricolândia          5              1      4   
 2    2023-10-01         Água Branca         29             40    -11   
 3    2023-10-01  Alagoinha do Piauí          1              3     -2   
 4    2023-10-01   Alegrete do Piauí          1              0      1   
 ..          ...                 ...        ...            ...    ...   
 185  2023-10-01               União         64            112    -48   
 186  2023-10-01              Uruçuí        311            198    113   
 187  2023-10-01    Valença do Piauí         55             30     25   
 188  2023-10-01       Várzea Grande          0              1     -1   
 189  2023-10-01         Wall Ferraz          0             10    -10   
 
     Salario_desligamento Salario_admissão  
 0                 1848.3          1693.62  
 1                 

In [10]:
caged_formatado_completo["faixaetária"] # verificando a disposição de um dos itens

,Período,faixaetária,Admissões,Desligamentos,Saldo,Salario_desligamento,Salario_admissão
0,2023-10-01,18 a 24 anos,3539,2344,1195,1193.68,1577.01
1,2023-10-01,25 a 29 anos,2385,2055,330,1456.04,1499.99
2,2023-10-01,30 a 39 anos,3518,3273,245,1611.14,1615.4
3,2023-10-01,40 a 49 anos,2033,1770,263,1695.51,1667.63
4,2023-10-01,50 a 64 anos,771,677,94,1745.92,1777.88
5,2023-10-01,Mais de 65 anos,22,45,-23,2700.99,2914.5


In [11]:
# Inserir análise dos salários no nosso dicionário
df_salários = analisar_salarios_aprimorado(caged_microdados_full, caged_dimensoes, ["Setores", "uf"])
caged_formatado_completo["valorsaláriofixo"] = df_salários

                                             Setores  \
0                                         Construção   
1      Serviços de transporte, armazenagem e correio   
2                                    Outros serviços   
3  Agricultura, pecuária, produção florestal, pes...   
4                           Alojamento e alimentação   
5                                    Indústria geral   
6  Informação, comunicação e atividades financeir...   
7                                Serviços domésticos   
8  Comércio, reparação de veículos automotores e ...   
9  Administração pública, defesa, seguridade soci...   

                                           Descrição  
0                                         Construção  
1      Serviços de transporte, armazenagem e correio  
2                                    Outros serviços  
3  Agricultura, pecuária, produção florestal, pes...  
4                           Alojamento e alimentação  
5                                    Indústria geral 

In [12]:
# salvando os arquivos
salvar_arquivos(caged_formatado_completo, "CAGED_outubro2")

In [13]:
# teste da analise combinada
df_detalhamento = analises_combinadas(caged_microdados, caged_dimensoes, grupos=["Setores"])
df_detalhamento.to_excel("./Tabelas/Setores.xlsx")

In [14]:
df_setor_por_muni = analises_combinadas(caged_microdados, caged_dimensoes, grupos=["município", "subclasse"])

idx = df_setor_por_muni.groupby("município")["Saldo"].idxmax()
df_setor_por_muni = df_setor_por_muni.loc[idx]
df_setor_por_muni.to_excel("./Tabelas/setor_por_muni2.xlsx")